In [1]:
!pip install -q arxiv openai pymsteams

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00


In [4]:
from openai import OpenAI
import arxiv
import pymsteams
import os
from google.colab import userdata


In [5]:
client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"))

In [9]:
def sum_Summary(result):
    system = """Bitte den Aufsatz immer nur in 3 Punkten auf Deutsch kurz zusammenfassen. Die Zusammenfassung sieht wie folgt aus:```
    Titel auf Deutsch:
    - Punkt1
    - Punkt2
    - Punkt3
    ```"""

    text = f"title: {result.title}\nbody: {result.summary}"
    response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {'role': 'system', 'content': system},
                    {'role': 'user', 'content': text}
                ],
                temperature=0.25,
            )
    summary = response.choices[0].message.content

    return summary


In [13]:
arxiv_client = arxiv.Client()

search = arxiv.Search(
    # Formulierung der Query siehe: https://info.arxiv.org/help/api/user-manual.html#query_details
    query = "cat:cs.AI",
    max_results = 1,
    sort_by=arxiv.SortCriterion.SubmittedDate,
    sort_order=arxiv.SortOrder.Descending,
)


In [14]:
newsBody = ""
for result in arxiv_client.results(search):
    sum = sum_Summary(result)
    authoren = ""
    for autor in result.authors:
        authoren += f" {autor.name},"
    artInfo = f"Artikeltitel: {result.title}\r \nKurze Zusammenfassung:\r \n{sum}\r \nAutor(en):{authoren}\r \nPubliziert: {result.published}\r \nLink: {result.entry_id}\r \n-------------"
    # print(artInfo)
    newsBody += artInfo


In [11]:
incoming_webhook_url = userdata.get("WEBHOOK_URL")

teams_obj = pymsteams.connectorcard(incoming_webhook_url)
teams_obj.title("Neuigkeit")
teams_obj.text(newsBody)
teams_obj.send()


True